This notebook references the following tutorials:

* building and saving an analysis tree
* loading and browsing an analysis tree

### Note:
The examples below call the low-level API and will be updated so that the higher-level `RWAnalyzer` API is called instead.

# Building and saving an analysis tree

Analysis tree with translocation data:

In [1]:

from tramway.helper import *

# preliminary: if trajectory_file does not exist, generate it
from tutorial import *
load_default_trajectories()

# load trajectory_file
trajectories = load_xyt(trajectory_file)

# the (trans-)location data is the top node of the analysis tree
analysis_tree = Analyses(trajectories)

print(analysis_tree)
#<Analyses <class 'pandas.core.frame.DataFrame'>>


<Analyses <class 'pandas.core.frame.DataFrame'>>


/home/flaurent/github/RWA-python/rwa/hdf5.py:185: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel # Panel has been flagged deprecated


Analysis tree with first partition:

In [2]:

# partition the locations into space cells
hexagonal_cells = tessellate(trajectories, 'hexagon')

# the partition is a first analysis; let's append it below trajs with label 'hexagonal mesh'
analysis_tree['hexagonal mesh'] = hexagonal_cells

# the two lines above could be combined the following way:
#hexagonal_cells = tessellate(analysis_tree, 'hexagon', label='hexagonal mesh')

print(analysis_tree)
#<class 'pandas.core.frame.DataFrame'>
#	'hexagonal mesh' <class 'tramway.tessellation.base.CellStats'>


<class 'pandas.core.frame.DataFrame'>
	'hexagonal mesh' <class 'tramway.tessellation.base.Partition'>


Analysis tree with second partition:

In [3]:

# make another partition
random_cells = tessellate(trajectories, 'random', cell_count=100)

# ...and add it to the analysis_tree below trajs with label 'random mesh'
analysis_tree['random mesh'] = random_cells

# similarly, the two lines above could be combined the following way:
#random_cells = tessellate(analysis_tree, 'random', cell_count=100, label='random mesh')

print(analysis_tree)
#<class 'pandas.core.frame.DataFrame'>
#	'hexagonal mesh' <class 'tramway.tessellation.base.CellStats'>
#	'random mesh' <class 'tramway.tessellation.base.CellStats'>


<class 'pandas.core.frame.DataFrame'>
	'hexagonal mesh' <class 'tramway.tessellation.base.Partition'>
	'random mesh' <class 'tramway.tessellation.base.Partition'>


Analysis tree with maps for the second partition:

In [4]:

# infer in D mode with random_cells
random_maps = infer(random_cells, 'D', sigma=0.01) # sigma = positioning error

# add the inferred maps below 'random mesh' with label 'D'
analysis_tree['random mesh']['D'] = random_maps

# again, the two lines above could be combined the following way:
#random_maps = infer(analysis_tree, 'D', sigma=0.01, input_label='random mesh', output_label='D')

print(analysis_tree)
#<class 'pandas.core.frame.DataFrame'>
#	'hexagonal mesh' <class 'tramway.tessellation.base.CellStats'>
#	'random mesh' <class 'tramway.tessellation.base.CellStats'>
#		'D' <class 'tramway.inference.base.Maps'>


<class 'pandas.core.frame.DataFrame'>
	'hexagonal mesh' <class 'tramway.tessellation.base.Partition'>
	'random mesh' <class 'tramway.tessellation.base.Partition'>
		'D' <class 'tramway.inference.base.Maps'>


Analysis tree with maps for the first partition:

In [5]:

# infer in D mode with snr extensions (for the bayes_factors plugin) and hexagonal_cells
hexagonal_maps = infer(hexagonal_cells, 'd.conj_prior', sigma=0.01)

# add the inferred maps below 'hexagonal mesh' with label 'D' as well
analysis_tree['hexagonal mesh']['D'] = hexagonal_maps

print(analysis_tree)
#<class 'pandas.core.frame.DataFrame'>
#	'hexagonal mesh' <class 'tramway.tessellation.base.CellStats'>
#		'D' <class 'tramway.inference.base.Maps'>
#	'random mesh' <class 'tramway.tessellation.base.CellStats'>
#		'D' <class 'tramway.inference.base.Maps'>


<class 'pandas.core.frame.DataFrame'>
	'hexagonal mesh' <class 'tramway.tessellation.base.Partition'>
		'D' <class 'tramway.inference.base.Maps'>
	'random mesh' <class 'tramway.tessellation.base.Partition'>
		'D' <class 'tramway.inference.base.Maps'>


Analysis tree with Bayes factor maps:

In [6]:

# compute the Bayes factor on these latter "maps"
bayes_factors = infer(analysis_tree, 'bayes_factor', sigma=0.01, input_label=('hexagonal mesh', 'D'), output_label='Bayes factors')
# here, the analysis tree is readily updated

print(analysis_tree)
#<class 'pandas.core.frame.DataFrame'>
#	'hexagonal mesh' <class 'tramway.tessellation.base.CellStats'>
#		'D' <class 'tramway.inference.base.Maps'>
#			'Bayes factors' <class 'tramway.inference.base.Maps'>
#	'random mesh' <class 'tramway.tessellation.base.CellStats'>
#		'D' <class 'tramway.inference.base.Maps'>

# save the analyses
save_rwa('my_analyses.rwa', analysis_tree)


Bayes factor calculations were skipped for them


<class 'pandas.core.frame.DataFrame'>
	'hexagonal mesh' <class 'tramway.tessellation.base.Partition'>
		'D' <class 'tramway.inference.base.Maps'>
			'Bayes factors' <class 'tramway.inference.base.Maps'>
	'random mesh' <class 'tramway.tessellation.base.Partition'>
		'D' <class 'tramway.inference.base.Maps'>


/home/flaurent/github/TRamWAy/tramway/inference/bayes_factors/group_by_sign.py:73: RuntimeWarning: invalid value encountered in greater_equal
  group_forces = 1 * (group_lg_B >= np.log10(B_threshold)) - \
/home/flaurent/github/TRamWAy/tramway/inference/bayes_factors/group_by_sign.py:74: RuntimeWarning: invalid value encountered in less_equal
  1 * (group_lg_B <= -np.log10(B_threshold))


# Loading and browsing an analysis tree

Loaded analysis tree:

In [7]:

from tramway.core.hdf5 import load_rwa
# or if you do not mind to load many modules:
#from tramway.helper import load_rwa

# load
analysis_tree = load_rwa('my_analyses.rwa')

# print the tree structure
print(analysis_tree)
#<class 'pandas.core.frame.DataFrame'>
#	'hexagonal mesh' <class 'tramway.tessellation.base.CellStats'>
#		'D' <class 'tramway.inference.base.Maps'>
#			'Bayes factors' <class 'tramway.inference.base.Maps'>
#	'random mesh' <class 'tramway.tessellation.base.CellStats'>
#		'D' <class 'tramway.inference.base.Maps'>


<class 'pandas.core.frame.DataFrame'>
	'hexagonal mesh' <class 'tramway.tessellation.base.Partition'>
		'D' <class 'tramway.inference.base.Maps'>
			'Bayes factors' <class 'tramway.inference.base.Maps'>
	'random mesh' <class 'tramway.tessellation.base.Partition'>
		'D' <class 'tramway.inference.base.Maps'>


Subtree from the `'random mesh'` node:

In [8]:

# retrieve some data
trajs = analysis_tree.data
hexagonal_cells = analysis_tree['hexagonal mesh'].data
hexagonal_maps = analysis_tree['hexagonal mesh']['D'].data
#...

# print a subtree
print(analysis_tree['random mesh'])
#<class 'tramway.tessellation.base.CellStats'>
#	'D' <class 'tramway.inference.base.Maps'>


<class 'tramway.tessellation.base.Partition'>
	'D' <class 'tramway.inference.base.Maps'>


`'random mesh'` partition:

In [9]:

# print some data
print(analysis_tree['random mesh'].data)
#location_count:  None
#@time_window:    duration:           1.0
#                 shift:              1.0
#                 time_dimension:     True
#@upper_bound:    [ 1.1  1.1]
#cell_index:      <class 'numpy.ndarray'>
#@partition:      metric:             euclidean
#                 knn:                10
#@method:         random
#bounding_box:    None
#@cell_count:     100
#points:          <class 'pandas.core.frame.DataFrame'>
#tessellation:    <class 'tramway.tessellation.random.RandomMesh'>
#@lower_bound:    [-0.1 -0.1]
#number_of_cells: 100
#@tessellation:   cell_count:         100
#                 avg_probability:    0.033927056827820185
#                 avg_location_count: 80


tessellation:    <class 'tramway.tessellation.random.RandomMesh'>
points:          <class 'pandas.core.frame.DataFrame'>
cell_index:      <class 'numpy.ndarray'>
location_count:  None
number_of_cells: 90
bounding_box:    None
@cell_count:     100
@method:         random
@tessellation:   avg_location_count: 80
                 avg_probability:    0.041109969167523124
                 cell_count:         100


`'random mesh'->'D'` maps:

In [10]:

print(analysis_tree['random mesh']['D'].data)
#mode:      d
#maps:      <class 'pandas.core.frame.DataFrame'>
#sigma:     0.01
#runtime:   0.11431741714477539
#features: ['diffusivity']


mode:     d
runtime:  0.18325233459472656
sigma:    0.01
features: ['diffusivity']
maps:     <class 'pandas.core.frame.DataFrame'>
